# Practical 1b: Hyperparameter Tuning

© Machine Learning Summer School - Telkom University

---


<table  class="tfo-notebook-buttons" align="left"><tr><td>
    
<a href="https://colab.research.google.com/github/adf-telkomuniv/MLSS2020_Telkom/blob/master/practical%201/MLSS2020TU%20-%20Practical%201b.ipynb" source="blank" ><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>
    
</td><td>
<a href="https://github.com/adf-telkomuniv/MLSS2020_Telkom/blob/master/practical%201/MLSS2020TU%20-%20Practical%201b.ipynb" source="blank" ><img src="https://i.ibb.co/6NxqGSF/pinpng-com-github-logo-png-small.png"></a>
    
</td></tr></table>

---
## Hyperparameter

Hyperparameters are variables whose value is set by the model designer before the learning process begins. These variables remain constant over the training process and directly impact the performance of your ML program.

hyperparameters might address model design questions such as:
* model (architectural) hyperparameters:
  * degree of polynomial features in linear model
  * number of layers and neurons in Neural Network
  * type of layers to use in Deep Learning
  * etc.

* training (algorithm) hyperparameters:
  * K value for Nearest Neighbors
  * learning rate for Gradient Descent
  * regularization
  * dropout value
  * etc.

This is in contrast to the weights in Artificial Neural Network, which are the values of parameters that are learned through the algorithm. 

---
## Optimization Approaches

There are several approaches to address this optimization problem. The most common of it are:
* Traditional Grid Search
 * commonly used to optimize width, depth, and types of layer in ANN
 * find the K value for KNN or Kernel type for SVMs
* Random Search
 * can be used to optimize the initial learning rate
* Bayesian opmitization
 * also used to optimize the initial learning rate
* Evolutionary optimization


<table>
  <tr><td  align="center">
    <img src="https://images.ctfassets.net/be04ylp8y0qc/1m1AB8NPTcKkuEaqg2Zgyg/e56d45a2c85b99820ead8159a280323f/hp_tuning_flow_1f745fd5e0ae8830804bab6a66e2c917_1000.png" width="80%" 
         alt="Hyperparameter tuning process">
  </td></tr>
  <tr><td align="center">
    <b>Figure:</b>Hyperparameter tuning process.<br/>
  </td></tr>
</table>


Grid Search optimization usually incorporated with K-Fold Cross-Validation scheme, while the rest only uses Hold-out validation. Especially in learning with huge dataset, prefer one validation fold to cross-validation. In most cases a single validation set of respectable size substantially simplifies the code base, without the need for cross-validation with multiple folds

For Neural Network and Deep Learning, Hyperparameter Tuning is usually only done with a little epoch in order to find a combination of hyperparameter that has the highest potential for the "actual" training process.

---
# [Part 0] Preparation

## Import Modules

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

%matplotlib inline


## Load Fashion MNIST Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(X_train[i+j*10], cmap='gray')
        ax[j,i].set_title(class_names[y_train[i+j*10]])
        ax[j,i].axis('off')
plt.show()

In [ ]:
X_val = X_train[-10000:,:]
y_val = y_train[-10000:]

X_train = X_train[:-10000, :]
y_train = y_train[:-10000]


In [ ]:
X_train = X_train.astype('float32') / 255.
X_val   = X_val.astype('float32') / 255.
X_test  = X_test.astype('float32') / 255.

---
#[Part 1] Keras Tuner

<img src = "https://i.ibb.co/4RyZpZ2/keras-tuner.png" height="200" align = "center">

Keras tuner is a hyperparameter optimization library (Hyperparameter tuner) built specifically for `tf.keras` with TensorFlow 2.0. It was introduced at the Google I/O 2019 event. 

This library is intended for AI practitioners, hypertuner algorithm creators, and model designers. It can be as simple as possible to build a model with a clean and easy API. Keras Tuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.

read more: [Documentation](https://keras-team.github.io/keras-tuner/), [github repository](https://github.com/keras-team/keras-tuner)


---
## 1 - Installation

First, install Keras Tuner 

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import kerastuner as kt

---
## 2 - Define Hyperparameter Range

In this example, we'll only use a Two-Layer Neural Network, and we're going to tune the layer width (number of hidden neuron), type of activation function, and the learning rate. Both hyperparameter range and the model need to be defined inside a method to feed into the tuner object. So in this part, we'll see the explanations first, then later the actual implementations.

### Layer Width

For this hyperparameter, we'll use grid choice to iterate from 128 to 256 with 64 increment. For that we use `Int()` method as follow

```python 
    ch_unit = hp.Int('units', min_value=128, max_value=256, step=64) 
```

### Activation Function
For the activation function, let's try `relu`, `tanh`, and `sigmoid` function. For that we use `Choice()` method as follow
```python
    ch_activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'], default='relu')
```

### Learning Rate
The learning rate represents how fast the learning algorithm progresses.The learning rate may, in fact, be the most important hyperparameter to configure for your model. Unfortunately, we cannot analytically calculate the optimal learning rate for a given model on a given dataset. Instead, a good (or good enough) learning rate must be discovered via trial and error. 

Arguably, it is actually not the best thing to tune learning rate using grid search. As mentioned by Bergstra and Bengio in [Random Search for Hyper-Parameter Optimization](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf), “randomly chosen trials are more efficient for hyper-parameter optimization than trials on a grid”. 

<table>
  <tr><td  align="center">
    <img src="https://cs231n.github.io/assets/nn3/gridsearchbad.jpeg" width="50%" 
         alt="Grid vs Random Layout.">
  </td></tr>
  <tr><td align="center">
    <b>Figure:</b> Grid vs Random Layout illustration from [Random Search for Hyper-Parameter Optimization](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).
  </td></tr>
</table>

Usually, the learning rate is chosen on a log scale. Intuitively, this is because learning rate have multiplicative effects on the training dynamics. This prior knowledge can be incorporated in the search through the setting of the sampling method:

```python
    ch_learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG', default=1e-3)
```

## 3 - Define Model

Now to define a function to build the model. We still use the usual Keras API, but feed in the choice for each hyperparameter. The function takes an argument hp from which we can use to sample the hyperparameters.


> <font color='red'>**EXERCISE**: </font> Define your classification model. 


In [ ]:
def build_model(hp):
  
    ch_unit          = ??
    ch_activation    = ??
    ch_learning_rate = ??

    # define model
    model = Sequential([
                Flatten(input_shape=(28,28)), 
                Dense(units=??, activation=??),
                Dense(10, activation='softmax')
            ])

    # compile model
    model.compile(
        optimizer = tf.keras.optimizers.Adam(??),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model

---
# [Part 2] Initialize The Tuner

The Keras Tuner has four tuners available - **RandomSearch**, **Hyperband**,  **Bayesian Optimization** and **Sklearn**. In this exercise, we'll focus on random search and Hyperband. We won't go into theory, but for Hyperband, its main idea is to optimize Random Search in terms of search time.

In here we can specify the model-building function, the name of the `objective` (metrics) to optimize, the total number of trials (`max_trials`) to test, and the number of models that should be built and fit for each trial (`executions_per_trial`). 

Similar to K-Fold Cross Validation, the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model.



In [ ]:
from kerastuner.tuners import RandomSearch, Hyperband

---
## 1 - Random Search Tuner
The most intuitive way to perform hyperparameter tuning is to randomly sample hyperparameter combinations and test them out. 

### Initialize Random Tuner
Here we will do the tuning experiment 4 times, with the best model selection metrics based on validation accuracy. The `max_trials` variable represents the number of hyperparameter combinations that will be tested by the tuner, while the `execution_per_trial` variable is the number of models that should be built and fit for each trial for robustness purposes.

In [ ]:
tuner_random = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='random_search',
    project_name='mnist_random'
    )


Once the model and the tuner are set up, we can view the search space summary

In [ ]:
tuner_random.search_space_summary()

### Run Random Tuning Process

Now we can start the Optimization for just 5 epochs. The arguments for the search method are the same as those used for `tf.keras.model.fit` 

Depend on how big your search space is, it might take a while.

In [ ]:
tuner_random.search(X_train, y_train,
             batch_size=256,
             epochs=5,
             validation_data=(X_val, y_val))

### View the Search Summary

Finally, the search results can be summarized and used

In [ ]:
tuner_random.results_summary()

### Show The Best Hyperparameter

In [ ]:
import tensorflow.keras.backend as K

# Retrieve the best model.
best_model_random = tuner_random.get_best_models(num_models=1)[0]
best_model_random.summary()

print('Best Initial Learning Rate =', K.eval(best_model_random.optimizer.lr))
print('Activation Function used   =', best_model_random.layers[1].get_config()['activation'],)

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(best_model_random, 
           to_file='best_random.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )


---
## 2 - Hyperband Tuner
Hyperband is an optimized version of random search which uses early-stopping to speed up the hyperparameter tuning process. The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. 


The main idea is to fit a large number of models for a small number of epochs and to only continue training for the models achieving the highest accuracy on the validation set. The `max_epochs` variable is the max number of epochs that a model can be trained for.

### Initialize Hyperband Tuner
Here we will do the tuning experiment 4 times, with the best model selection metrics based on validation accuracy.

In [ ]:
tuner_hyperband = Hyperband(
    build_model,
    max_epochs=5,
    objective='val_accuracy',
    executions_per_trial=2,
    directory='hyperband',
    project_name='mnist_hyperband'
)



Once the model and the tuner are set up, we can view the search space summary

In [ ]:
tuner_hyperband.search_space_summary()

### Run Hyperband Tuning Process

Now we can start the Optimization for just 5 epochs. 

In [ ]:
tuner_hyperband.search(X_train, y_train,
             batch_size=256,
             epochs=5,
             validation_data=(X_val, y_val))

### View the Search Summary

Finally, the search results can be summarized and used

In [ ]:
tuner_hyperband.results_summary()

### Show The Best Hyperparameter

In [ ]:
# Retrieve the best model.
best_model_hyperband = tuner_hyperband.get_best_models(num_models=1)[0]
best_model_hyperband.summary()

print('Best Initial Learning Rate =', K.eval(best_model_hyperband.optimizer.lr))
print('Activation Function used   =', best_model_random.layers[1].get_config()['activation'])

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(best_model_hyperband, 
           to_file='best_hyperband.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='LR',
           dpi=70
          )


---
## 3 - Evaluate Best Model

In [ ]:
# loss, accuracy = best_model_random.evaluate(X_test, y_test)

loss, accuracy = best_model_hyperband.evaluate(X_test, y_test)

print('Validation Accuracy =',accuracy)

---
### View Result
Now to visualize some of the model's predictions:

In [ ]:
# y_pred = best_model_random.predict(X_test, verbose=0)

y_pred = best_model_hyperband.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
fig, ax = plt.subplots(2,8,figsize=(22,6))
fig.subplots_adjust(hspace=0.2, wspace=0.4)

for j in range(0,2):
    for i in range(0, 8):

        img_index = np.random.randint(0, 10000)
        ax[j,i].imshow(X_test[img_index], cmap='gray')

        actual_label    = int(y_test[img_index])
        predicted_label = int(y_pred[img_index])

        color = 'red'
        if actual_label == predicted_label:
            color = 'green'

        ax[j,i].set_title("Actual: {} ({})\n Pred: {} ({})".format(
            actual_label, class_names[actual_label], predicted_label, class_names[predicted_label]
            ), color=color)
        ax[j,i].axis('off')
plt.show()

---

# Congratulation

<font size=5> You've Completed Practical 1b</font>

<p>Copyright &copy;  <a href=https://www.linkedin.com/in/andityaarifianto/>2020 - ADF</a> </p>